In [1]:
import pdfplumber as pdf
import os
import pandas as pd

In [2]:
# shows files in working directory
path = "C:/Users/Admin/Desktop/LettoreFatture"
files = os.listdir(path)
print(files)

['.git', '.gitignore', '.ipynb_checkpoints', 'main.ipynb', 'output.xlsx', 'testfile.pdf', 'testfile2.pdf', 'testfile3.pdf', 'testfile4.pdf']


In [3]:
# opens a pdf object
files = [files[i] for i in range(len(files)) if files[i].endswith('pdf') == True]
open_files = [pdf.open(files[i]) for i in range(len(files))]
pages = [open_files[i].pages for i in range(len(files))]

In [5]:
words_dict = []
text = ''
# for every page the text is extracted
for k in range(len(pages)):
    for i in range(len(pages[0])):
        text += pages[k][i].extract_text()
    words_dict.append(text)
    text = ''
    
# we split the text at each endline
lines = []
for i in range(len(words_dict)):
    lines.append(words_dict[i].split(sep = '\n'))

# we separate keys from values in every entry
# and delete lines who don't contain values
split = []
entries = []

for k in range(len(lines)):
    split.append([lines[k][i].split(sep = ':') for i in range(len(lines[k]))])
    entries.append([split[k][i] for i in range(len(split[k])) if len(split[k][i]) == 2])

In [6]:
# we store separately keys and values
# and put them in a dictionary
keys = [list(list(zip(*entries[i]))[0]) for i in range(len(entries))]
values = [list(list(zip(*entries[i]))[1]) for i in range(len(entries))]


new_dict = [{key: value for key,
            value in zip(keys[i], values[i])} for i in range(len(entries))]

# converts dictionary to dataframe and to excel
df = [pd.DataFrame(new_dict[i], index = [0]) for i in range(len(new_dict))]

In [22]:
final = df[0]
for k in range(len(df)-1):
    final = pd.merge(final,df[k+1], 'outer')

In [24]:
final.to_excel("output.xlsx")